# Analysing the impact of COVID on special educational needs - notebook 3: change by ESL numbers

Data on pupil numbers is at https://explore-education-statistics.service.gov.uk/find-statistics/school-pupils-and-their-characteristics. The raw data can be downloaded, queries can be made, or particular queries re-loaded.  

A zip file can be accessed from the 'Download all data' button at https://explore-education-statistics.service.gov.uk/find-statistics/school-pupils-and-their-characteristics#explore-data-and-files

This would allow us not only to put SEN figures into the context of pupil numbers (alternative hypothesis: any rise in SEN numbers is simply due to a rise in pupil numbers for whom English is a second language)


## Install the packages we need

In [ ]:
#load rmagic to be able to run R
#from https://towardsdatascience.com/how-to-use-r-in-google-colab-b6e02d736497
%load_ext rpy2.ipython

In [ ]:
%%R
#install the tidyverse package
install.packages('tidyverse')
library('tidyverse')
#install the downloader package: https://cran.r-project.org/web/packages/downloader/index.html
install.packages("downloader")
library(downloader)

(as ‘lib’ is unspecified)







	‘/tmp/RtmpJYgCGV/downloaded_packages’



── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


(as ‘lib’ is unspecified)







	‘/tmp/RtmpJYgCGV/downloaded_packages’



In [ ]:
%%R
install.packages('stringi')
library(stringi)

(as ‘lib’ is unspecified)







	‘/tmp/RtmpJYgCGV/downloaded_packages’



## Import the zip file, extract the data

The data is published in a zip file that can be accessed from the 'Download all data' button at https://explore-education-statistics.service.gov.uk/find-statistics/school-pupils-and-their-characteristics#explore-data-and-files


In [ ]:
%%R
#store the URL for the data zip file which is found by right-clicking on 
#'Download all data' at https://explore-education-statistics.service.gov.uk/find-statistics/school-pupils-and-their-characteristics#explore-data-and-files
zipurl <- "https://content.explore-education-statistics.service.gov.uk/api/releases/cf516998-1dc1-411d-8225-13f6320547fb/files"


In [ ]:
%%R
#download the zip file from the url
downloader::download(zipurl, dest="datasets.zip", mode="wb") 
#unzip it 
unzip ("datasets.zip", exdir = "./")


  cannot open URL 'https://content.explore-education-statistics.service.gov.uk/api/releases/cf516998-1dc1-411d-8225-13f6320547fb/files'


 




Error in download.file(url, method = method, ...) : 
  cannot open URL 'https://content.explore-education-statistics.service.gov.uk/api/releases/cf516998-1dc1-411d-8225-13f6320547fb/files'


RInterpreterError: ignored

In [ ]:

%%R 
#import the data extracted from the zip
#this is the name in 2022
data22 <- "data/spc_pupils_ethnicity_and_language_.csv"
data <- readr::read_csv(data22)

data

## `filter()` the data to what we need

This is a very large dataframe, and we don't need all of it. Firstly, we're only interested in primary schools.

In [ ]:
%%R
#filter to where column values match specified string
slcdf <- filter(data, phase_type_grouping == "State-funded primary")
print(slcdf)

### Filter: local authorities only

We also only want to look at data for local authorities. 

In [ ]:
%%R
#show the unique values and their frequency
table(slcdf['geographic_level'])

In [ ]:
%%R
#filter to where column values match specified string
slcdf_filter4 <- filter(slcdf, geographic_level == "Local authority")
print(slcdf_filter4)

### Filter: All ethnicities only

There are separate counts for each ethnicity, as well as a total. We only need the totals.

In [ ]:
%%R
#count the unique values in the time_identifier column
print(table(slcdf_filter4['ethnicity']))

In [ ]:
%%R
#filter to where column values match specified string
slcdf_filter4 <- filter(slcdf_filter4, ethnicity == "Total")
print(slcdf_filter4)

### Filter: Language

There are four categories for language. We only need the row specifying how many are other than English.

In [ ]:
%%R
#count the unique values in the time_identifier column
print(table(slcdf_filter4['language']))

In [ ]:
%%R
#filter to where column values match specified string
slcdf_filter4 <- filter(slcdf_filter4, language == "Known or believed to be other than English")
print(slcdf_filter4)

## Remove columns

We also have a number of columns we don't need.

In [ ]:
%%R
#show the columns
colnames(slcdf_filter4)

Some columns only have one value

In [ ]:
%%R
#count the unique values in the time_identifier column
print(table(slcdf_filter4['time_identifier']))
#repeat for the geographic level column
print(table(slcdf_filter4['geographic_level']))
#repeat for country_name
print(table(slcdf_filter4['country_name']))
#repeat for country_code
print(table(slcdf_filter4['country_code']))
#repeat for phase_type_grouping
print(table(slcdf_filter4['phase_type_grouping']))
#repeat for phase_type_grouping
print(table(slcdf_filter4['ethnicity']))

We remove that by using `select()` and a minus before the column we want to exclude.

Note: we remove `headcount` because we are only interested in the percentage of pupils, and otherwise the `spread()` function below does not work.

In [ ]:
%%R
#remove the specified columns
slcdf_col_filter1 <- select(slcdf_filter4, 
                            -c(time_identifier, 
                               geographic_level,
                               country_name,
                               country_code,
                               phase_type_grouping,
                               ethnicity,
                               headcount))
slcdf_col_filter1

## Reshape long to wide using `spread()`

We now reshape to make that column of years into the column headings, so there's only one row per LA, and we can more easily calculate year on year change.

Note that R doesn't like number-only column names, so it puts each one inside the code accent: `

In [ ]:
%%R
#specify we want to convert the time_period column values to column names 
#and insert the values from the sum_of_yr1 column underneath
esl_wide <- slcdf_col_filter1 %>% spread(time_period, percent_of_pupils)
#show the results
esl_wide[c(4,7,8,9,10,11,12,13)]

## Calculate year on year changes

Now we can add new columns with the year on year changes.

In [ ]:
%%R
#remind ourselves of the column names
colnames(esl_wide)

In [ ]:
%%R
#loop through the column names from 8th to 13th position
for (i in seq(8,13)){
    print(i)
    thisyr <- colnames(esl_wide)[i]
    previousyr <- colnames(esl_wide)[i-1]
    print(thisyr)
    print(previousyr)
    #subtract the previous year from the current one to get the change - and store 
    YOYchange <- esl_wide[thisyr] - esl_wide[previousyr]
    YOYpercChange <- YOYchange/esl_wide[previousyr]
    #extract the two digits for the later year
    toyr <- substr(thisyr,5,6)
    fromyr <- substr(previousyr,5,6)
    #create column names from these
    colname <- paste0("YOY",fromyr,"to",toyr)
    perccolname <- paste0("YOYperc",fromyr,"to",toyr)
    #create a new column with that name and the values calculated
    esl_wide[colname] <- YOYchange
    esl_wide[perccolname] <- YOYpercChange
}

In [ ]:
%%R
esl_wide

## Export results

Let's export as a CSV.

In [ ]:
%%R
#export as a CSV
write.csv(esl_wide, "esl_totals.csv")

## Merge with the SEN analysis

A glance already tells us that there's been no pupil numbers increase to account for the increase in demand for SEN. 

But we still need to merge it to put those increases into context.

We've uploaded the results of the analysis from the first notebook to Google Drive and published as a CSV, which is imported below. The data can also be generated by the other notebook and uploaded to the Files area on the left if needed.

In [ ]:
%%R
#store the URL for the CSV
analysiscsvurl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTRm8tw8O_QO_o4bk4Y_3sUlX0N5ukNrCeHj08RyShS3cAEmKTZdvB8g48zDHbl8l_dmDtjOUUrB12L/pub?gid=814230511&single=true&output=csv"
#import the CSV into a dataframe
analysisdata <- readr::read_csv(analysiscsvurl)
analysisdata

Let's hope the LA names are consistent... if they are we should get a dataframe with 155 rows (the amount in the analysis CSV - there are 156 in the pupil numbers dataframe).

In [ ]:
%%R
#merge the two dataframes on the la_name colum - all = F makes it an inner join
merged_data <- merge(analysisdata, esl_wide, by = "la_name", all = F, suffixes = c("_language","_esl"))
#check results
print(nrow(merged_data))
print(colnames(merged_data))

## Export merged results

Let's export again.

In [ ]:
%%R
#export as a CSV
write.csv(merged_data, "merged_sen_esl_data.csv")

## Compare SEN percentages with ESL percentages

If SEN changes are 'caused' by a rise in pupils with English as a second language then there should be some sort of relationship. 

In [ ]:
%%R
print(colnames(merged_data))

In [ ]:
%%R
#create a simple data frame with the 2021 figures
percs2021 <- merged_data[,c(21,43)]

In [ ]:
%%R
#calculate a correlation coefficient, removing pairs where one number is missing
#https://www.statmethods.net/stats/correlations.html
cor(percs2021, use="pairwise.complete.obs", method="pearson")

In [ ]:
%%R
#create a simple data frame with the 2021 figures and the 2020 ESL change
percs20to21 <- merged_data[,c(21,42)]
#calculate a correlation coefficient, removing pairs where one number is missing
#https://www.statmethods.net/stats/correlations.html
cor(percs20to21, use="pairwise.complete.obs", method="pearson")

In [ ]:
%%R
#create a simple data frame with the 2021 figures and the 2020 ESL change
percs19to21 <- merged_data[,c(21,41)]
#calculate a correlation coefficient, removing pairs where one number is missing
#https://www.statmethods.net/stats/correlations.html
cor(percs19to21, use="pairwise.complete.obs", method="pearson")

None of these correlations are even approaching significant.